# 对CIFAR的实操

模型结构图片如下所示：

![CIFAR_nn_model_structure](./md_assets/CIFAR_model_structure.jpg)

<img src='./md_assets/CIFAR_model_structure.jpg', zoom='50%'></img>

通过两种方式实现模型。



In [9]:
import torch
from torch.nn import Conv2d, MaxPool2d, MaxPool2d, Linear, Flatten, Module
from torch.utils.tensorboard import SummaryWriter


## 逐层定义网络模型

In [10]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 输入数据格式为(3,32,32) 第一个值为通道数目
        # 已知输入格式和输出格式，通过卷积公式计算padding和stride
        self.conv1 = torch.nn.Conv2d(3, 32, 5, padding=2)
        # 第一次卷积后的格式为(32,32,32)
        self.maxpooling1 = torch.nn.MaxPool2d(kernel_size=2)
        # 池化后的格式为(32,16,16)
        self.conv2 = torch.nn.Conv2d(32, 32, 5, padding=2)
        # 卷积后的尺寸是(32,16,16)
        self.maxpooling2 = torch.nn.MaxPool2d(kernel_size=2)
        # 池化后的尺寸是(32,8,8)
        self.conv3 = torch.nn.Conv2d(32, 64, 5, padding=2)
        # 卷积后的尺寸是(64,8,8)
        self.maxpooling3 = torch.nn.MaxPool2d(kernel_size=2)
        # 池化后是(64,4,4)
        self.flatten = torch.nn.Flatten()
        self.linear1 = torch.nn.Linear(in_features=1024, out_features=64)
        self.linear2 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpooling1(x)
        x = self.conv2(x)
        x = self.maxpooling2(x)
        x = self.conv3(x)
        x = self.maxpooling3(x)
        x = self.flatten(x)
        x = self.linear1(x)
        y = self.linear2(x)
        return y


如果对于数据的格式不是很确定，可以逐层测试模型的输出结果。


In [11]:
n = Net()
print(f'>> print Net obj: n = {n}')


# 检验网络（尺寸）数据是否合理
input = torch.ones((64, 3, 32, 32))
output = n(input)
print(f'>> output.shape: {output.shape}')
# print(f'>> output: {output}')


>> print Net obj: n = Net(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpooling2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpooling3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=1024, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=10, bias=True)
)
>> output.shape: torch.Size([64, 10])


## 定义Sequential

通过定义一个Sequential，简化网络层定义以及forward函数的定义。



In [12]:
class Net(Module):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.Sequential(
            Conv2d(3,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10)
        )
    
    def forward(self, x):
        y = self.model(x)
        return y


通过Tensorboard也可以进行可视化


In [13]:
writer = SummaryWriter(log_dir='../logs')

x = torch.ones((64, 3, 32, 32))
y = n(x)
print(f'>> y.shape: {y.shape}')

writer.add_graph(n, x)
writer.close()


>> y.shape: torch.Size([64, 10])
